In [1]:
!pip install requests beautifulsoup4 pandas


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

BASE_URL = 'https://sxyprn.com'

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
count = 0


In [3]:
def get_posts_from_page(page_link):
    global count
    response = requests.get(page_link)
    soup = BeautifulSoup(response.text, "html.parser")
    search_results = soup.find_all(class_="search_results")
    post_links = [BASE_URL +a["href"] for result in search_results for a in result.find_all("a", href=True) if a["href"].startswith("/post/")]
    post_links = list(set(post_links))
    print(count)
    count += 1
    return post_links

In [4]:
def get_posts_from_category(category_link):
    print('category_link:' + category_link)
    post_links = []
    response = requests.get(category_link)
    soup = BeautifulSoup(response.text, "html.parser")
    searches_container = soup.find('div', {'id': 'center_control'})
    pagination_elements = searches_container.find_all('a')
    pagination_element_links = [BASE_URL + elem.get('href') for elem in pagination_elements]

    # Use ThreadPoolExecutor to run get_posts_from_page concurrently
    with ThreadPoolExecutor(max_workers=300) as executor:  # Adjust max_workers as needed
        results = executor.map(get_posts_from_page, pagination_element_links)

    for result in results:
        post_links += result
        df = pd.DataFrame(list(set(post_links)), columns=["links"])
        df.to_csv(category_link.replace("/", "_") + "links.csv", index=False) 
        print(post_links)
        
    
    
    return post_links

In [5]:
def process_page(i):
    """ Fetch category links from a page and get posts from those categories """
    post_links = []
    response = requests.get(BASE_URL + f'/searches/{i}.html')
    soup = BeautifulSoup(response.text, "html.parser")
    
    searches_container = soup.find('div', {'id': 'searches_container'})
    if not searches_container:
        return []

    category_link_elements = searches_container.find_all('a')
    category_links = [BASE_URL + elem.get('href') for elem in category_link_elements]

    with ThreadPoolExecutor(max_workers=10) as executor:
        results = executor.map(get_posts_from_category, category_links)

    for result in results:
        post_links += result

    return post_links

def get_post_links():
    post_links = []
    with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust max_workers as needed
        results = executor.map(process_page, range(0, 26701, 150))

    for result in results:
        post_links += result

    return post_links
            

In [ ]:
get_post_links()

category_link:https://sxyprn.com/Anal.html?sm=trending&trends=1
category_link:https://sxyprn.com/French.html?sm=trending&trends=2
category_link:https://sxyprn.com/MILF.html?sm=trending&trends=3
category_link:https://sxyprn.com/BigTits.html?sm=trending&trends=4
category_link:https://sxyprn.com/Lesbian.html?sm=trending&trends=5
category_link:https://sxyprn.com/Onlyfans.html?sm=trending&trends=6
category_link:https://sxyprn.com/Indian.html?sm=trending&trends=7
category_link:https://sxyprn.com/Casting.html?sm=trending&trends=8
category_link:https://sxyprn.com/Creampie.html?sm=trending&trends=9
category_link:https://sxyprn.com/Sex-And.html?sm=trending&trends=10
category_link:https://sxyprn.com/CumSwapping.html?sm=trending&trends=751
category_link:https://sxyprn.com/Lena-Coxx.html?sm=trending&trends=752
category_link:https://sxyprn.com/Sweet-Vickie.html?sm=trending&trends=754
category_link:https://sxyprn.com/Milf-Pov.html?sm=trending&trends=753
category_link:https://sxyprn.com/FanBus.html?sm